In [ ]:
# Set the folder paths so that it can be used later
silver_folder_path = 'abfss://ws-container@bogomarketingdl.dfs.core.windows.net/silver'
gold_folder_path = 'abfss://ws-container@bogomarketingdl.dfs.core.windows.net/gold'

In [ ]:
from pyspark.sql import functions as F

In [ ]:
def ingest_table(table_name):
    df = spark.read.parquet(f"{silver_folder_path}/{table_name}")
    # Create the 'day' column by dividing 'time' by 24
    df = df.withColumn('day', ((F.col('time') / 24)+1).cast('int'))
    df = df.dropDuplicates()
    df.write.mode("overwrite").partitionBy("day","time")\
    .format("parquet").saveAsTable(f"gold_reward_program_spark.{table_name}")

In [ ]:
table_names = ['transaction','order_received','order_viewed','order_completed']

foreach table_name in table_names:
    ingest_table(table_name)